In [4]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re
import os

In [5]:
def fetch_data():
    url = 'https://droneportal.or.kr/subList/22000000153'
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = urllib.request.Request(url, headers=headers)
    try:
        html_data = urllib.request.urlopen(req).read().decode('utf-8')
    except Exception as e:
        print("웹 페이지 요청 실패:", e)
        exit()
    soup = BeautifulSoup(html_data, 'html.parser')
    tables = soup.find_all('table')
    target_table = None
    for table in tables:
        if "UA" in table.get_text():
            target_table = table
            break
    if target_table is None:
        print("대상 테이블을 찾을 수 없습니다.")
        exit()
    rows = target_table.find_all("tr")
    data = []
    for row in rows:
        cells = row.find_all(["td", "th"])
        cell_texts = [cell.get_text(strip=True) for cell in cells][:4]
        while len(cell_texts) < 4:
            cell_texts.append(None)
        # 행의 어느 셀에라도 "UA"와 숫자가 있으면 포함
        if cell_texts and any(re.search(r'\bUA\s*\d+\b', cell) for cell in cell_texts if cell):
            data.append(cell_texts)
    columns = ["구역 코드", "위치", "수평범위", "수직범위"]
    return pd.DataFrame(data, columns=columns)

In [6]:
# csv파일 저장 경로
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
top_dir = os.path.dirname(parent_dir)
data_dir = os.path.join(top_dir, '데이터')
NE_table_path = os.path.join(data_dir, r'원본(크롤링)\original_NE.csv')

In [8]:
# csv파일 저장
original_NE = fetch_data()
original_NE.to_csv(NE_table_path, encoding='utf-8', index=False)

OSError: Cannot save file into a non-existent directory: '/데이터'